In [8]:
import numpy as np
import pandas as pd

import os 
import tqdm 
# add a path to the sys.path
import sys


NameError: name 'lang' is not defined

In [26]:

import sys, os
from tqdm import tqdm
import multiprocessing
from jiwer import compute_measures
from zhon.hanzi import punctuation
import string
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline
import soundfile as sf
import scipy
import zhconv
from funasr import AutoModel
from datasets import load_dataset


lang = "en"
device = "cuda:0"
punctuation_all = punctuation + string.punctuation + '.,!?'
def process_one(hypo, truth):
    raw_truth = truth
    raw_hypo = hypo

    for x in punctuation_all:
        if x == '\'':
            continue
        truth = truth.replace(x, '')
        hypo = hypo.replace(x, '')
    truth = truth.replace('[laughter]','')
    hypo = hypo.replace('[laughter]','')
    
    truth = truth.replace('  ', ' ')
    hypo = hypo.replace('  ', ' ')
    truth = truth.lower()
    hypo = hypo.lower()
    
    #print("hypo", hypo, "truth", truth)

    if lang == "zh":
        truth = " ".join([x for x in truth])
        hypo = " ".join([x for x in hypo])
    elif lang == "en":
        truth = truth.lower()
        hypo = hypo.lower()
    else:
        raise NotImplementedError

    measures = compute_measures(truth, hypo)
    ref_list = truth.split(" ")
    wer = measures["wer"]
    subs = measures["substitutions"] / len(ref_list)
    dele = measures["deletions"] / len(ref_list)
    inse = measures["insertions"] / len(ref_list)
    
    # print("hyp", hypo,  "truth", truth, "WER", wer, "subs", subs, "dele", dele, "inse", inse)
    return (truth, hypo, wer, subs, dele, inse)


In [34]:


file_list = ["/exp/leying.zhang/Fisher-dataset-test/2spk_soundstorm_simu/covosingle/wer_results_score",
             "/exp/leying.zhang/Fisher-dataset-test/2spk_soundstorm_simu/covomix/wer_results_score",
             "/exp/leying.zhang/Fisher-dataset-test/2spk_soundstorm_simu/soundstorm_dongmei20240807/setting_3_steps_5_gumbleSample_False/steps_5_gumbleSample_False/wer_results_score",
             "/exp/leying.zhang/Fisher-dataset-test/2spk_soundstorm_simu/ground_truth_after_vocoder/wer_results_score"]

gt_trans = {}
with open('/exp/leying.zhang/Fisher-dataset-test/1spk_monologue/test/soundstorm_evaluation_pairs_eval.txt') as f:
    labels = f.readlines()
    for l in labels:
        l = l.strip().split('\t')
        utt = l[0].split('/')[-1]
        trans = l[-1]
        gt_trans[utt] = trans 

for file in file_list:
    wers = []
    with open(file) as f:
        labels = f.readlines()
        for l in labels:
            l = l.strip().split('\t')
            utt = l[0].split('/')[-1]
            trans_predict = l[-4]
            if len(trans_predict.split(' ')) < len(gt_trans[utt].split(' '))-2 or len(trans_predict.split(' ')) > len(gt_trans[utt].split(' '))+2:
                continue
            raw_truth, raw_hypo, wer, subs, dele, inse = process_one(trans_predict, gt_trans[utt])
            wers.append(float(wer))
    print(f"File: {file}, WER: {np.mean(wers)*100}")

File: /exp/leying.zhang/Fisher-dataset-test/2spk_soundstorm_simu/covosingle/wer_results_score, WER: 13.671373835328124
File: /exp/leying.zhang/Fisher-dataset-test/2spk_soundstorm_simu/covomix/wer_results_score, WER: 15.312884971868131
File: /exp/leying.zhang/Fisher-dataset-test/2spk_soundstorm_simu/soundstorm_dongmei20240807/setting_3_steps_5_gumbleSample_False/steps_5_gumbleSample_False/wer_results_score, WER: 16.050754449580342
File: /exp/leying.zhang/Fisher-dataset-test/2spk_soundstorm_simu/ground_truth_after_vocoder/wer_results_score, WER: 14.006949736341634


In [22]:
gt_trans

{'': "okay oh you do how many packs do you smoke in a day from my window and i watched the building fall and as soon as it fell i got in my car and i left i didn't even say well goodnight you know i'll see you guys tomorrow or is it okay to leave i left okay my opinion is i'm glad they did 'cause i'm not a smoker"}